<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# ICML - Run GriddedCCPUnet  Shared or Not

In [1]:
cd ..

/private/home/yannd/projects/Neural-Process-Family


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
    
import neuralproc

In [3]:
import submitit
from train_imgs import main, parse_arguments

log_folder="logs/%j"

In [4]:
def repeat_list(l, n):
    return [i for i in l for _ in range(n)]

class Run:
    def checkpoint(self, args):
        """Resubmits the same callable with the same arguments but makes sure continnue from last chckpnt."""
        args.is_continue_train=True 
        return submitit.utils.DelayedSubmission(self, args)
    
    def __call__(self, args):
        job_env = submitit.core.utils.JobEnvironment()
        args.starting_run = args.starting_run * job_env.num_tasks + job_env.local_rank
        return main(args)

In [5]:
datasets_32 = [("cifar10","--is-augment"),("svhn",""),("mnist","")]
models = [("GridedCCPUnet", "--name GridedCCPUnet_large --n-blocks 7 --r-dim 256"),
          ("GridedCCPUnetShared", "--name GridedCCPUnetShared_large --n-blocks 7 --r-dim 256"),
          ("GridedCCPUnet", "--name GridedCCPUnet_small --n-blocks 5 --r-dim 64 "),
          ("GridedCCPUnetShared", "--name GridedCCPUnetShared_small --n-blocks 5 --r-dim 64 ")]

args_32_ccp = [parse_arguments(f"{m} {d} --starting-run {s}  --chckpnt-dirname results/icml_dev/ {d_kwargs} {m_kwargs} --max-epochs 2 --no-retrain --clf labelspread".split())
                for s in range(1)
                for d,d_kwargs in datasets_32
                for m,m_kwargs in models]

In [6]:
executor_32_ccp=submitit.SlurmExecutor(folder=log_folder, max_num_timeout=2)
executor_32_ccp.update_parameters(num_gpus=1, 
                                   cpus_per_task=10, 
                                   mem='32GB',
                                   ntasks_per_node=2,
                                  time=20,  
                                  partition="dev"
                                  )

In [ ]:
main(args_32_ccp[0])

Files already downloaded and verified
Files already downloaded and verified

--- Loading cifar10/GridedCCPUnet_large/run_0 ---



/private/home/yannd/projects/Neural-Process-Family/neuralproc/predefined/mlp.py:63: UserWarning: hidden_size=64 larger than output=6 and input=32. Setting it to 32.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))


cifar10/GridedCCPUnet_large/run_0 | best epoch: 2 | train loss: -2.7013 | valid loss: -2.923 | test log likelihood: 2.9447
transformed data
> /private/home/yannd/projects/Neural-Process-Family/train_imgs.py(380)main()
-> clf_datasets = reduce_datasets(transformed_dataset, n_labels=dict(train=n_label), seed=seed)


In [29]:
job_32_ccp = executor_32_ccp.map_array(Run(), args_32_ccp)

In [30]:
############################################################

In [38]:
job_32_ccp

[SlurmJob<job_id=20444762_0, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=20444762_1, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=20444762_2, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=20444762_3, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=20444762_4, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=20444762_5, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=20444762_6, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=20444762_7, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=20444762_8, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=20444762_9, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=20444762_10, task_id=None, state="COMPLETED">,
 SlurmJob<job_id=20444762_11, task_id=None, state="COMPLETED">]

In [39]:
for j in job_32_ccp:
    print("--------------------------------")
    print(j.stdout())

--------------------------------
submitit INFO (2019-11-27 02:44:17,956) - Starting with JobEnvironment(job_id=20444762_0, hostname=learnfair0593, local_rank=0(2), node=0(1), global_rank=0(2))
submitit INFO (2019-11-27 02:44:17,957) - Loading pickle: /private/home/yannd/projects/Neural-Process-Family/logs/20444762_0/20444762_0_submitted.pkl
Files already downloaded and verified
Files already downloaded and verified

--- Training cifar10/GridedCCPUnet_large/run_0 ---

  epoch    train_loss    valid_loss    cp      dur
-------  ------------  ------------  ----  -------
      1       -1.9275       -2.6155     +  82.1843
      2       -2.7013       -2.9230     +  81.3522
Re-initializing module.
Re-initializing optimizer.
cifar10/GridedCCPUnet_large/run_0 | best epoch: 2 | train loss: -2.7013 | valid loss: -2.923 | test log likelihood: 2.9447
submitit INFO (2019-11-27 02:47:54,369) - Job completed successfully

submitit INFO (2019-11-27 02:44:17,956) - Starting with JobEnvironment(job_id=20

In [26]:
for j in job_32_ccp:
    print("--------------------------------")
    print(j.stderr())

--------------------------------
/private/home/yannd/projects/Neural-Process-Family/neuralproc/predefined/mlp.py:63: UserWarning: hidden_size=64 larger than output=6 and input=32. Setting it to 32.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))

100%|█████████▉| 2967/2970 [01:30<00:00, 49.42it/s, valid_loss=-3.3] 
                                                                    

100%|█████████▉| 2966/2970 [01:29<00:00, 48.36it/s, valid_loss=-3.34]
                                                                     

100%|█████████▉| 2966/2970 [01:29<00:00, 49.06it/s, valid_loss=-3.47]
                                                                     

100%|█████████▉| 2967/2970 [01:28<00:00, 39.43it/s, valid_loss=-3.62]
                                                                     

100%|█████████▉| 2967/2970 [01:29<00:00, 47.63it/s, valid_loss=-3.64]
                                                                     

 99%|████████

/private/home/yannd/projects/Neural-Process-Family/neuralproc/predefined/mlp.py:63: UserWarning: hidden_size=64 larger than output=6 and input=32. Setting it to 32.
  warnings.warn(txt.format(hidden_size, output_size, input_size, self.hidden_size))

100%|█████████▉| 5781/5782 [02:37<00:00, 43.92it/s, valid_loss=-3.72]
                                                                     

100%|█████████▉| 5780/5782 [02:37<00:00, 44.33it/s, valid_loss=-3.08]
                                                                     

100%|█████████▉| 5778/5782 [02:34<00:00, 57.24it/s, valid_loss=-3.6] 
                                                                    

100%|█████████▉| 5778/5782 [02:34<00:00, 46.14it/s, valid_loss=-3.38]
                                                                     

100%|█████████▉| 5780/5782 [02:32<00:00, 43.71it/s, valid_loss=-3.82]
                                                                     

100%|█████████▉| 312/313 [00:05<00:00, 56.47it

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [16]:
IS_CANCEL = False

if IS_CANCEL:
    for j in job_32_ccp:
        j.cancel()